In [1]:
from selenium import webdriver
import pandas as pd
import time
import os
import zipfile
import shutil

In [2]:
# 屏幕最大化，且指定下载目录
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")

prefs = {"profile.default_content_settings.popups": 0,
         "download.default_directory": r"D:\Program Files (x86)\百度云同步盘\Dropbox\-E·J- 2014.5.1\2016.12.15 店小秘数据分析\2017.10.24 各渠道运费明细\燕文\\", # IMPORTANT - ENDING SLASH V IMPORTANT
         "directory_upgrade": True}
options.add_experimental_option("prefs", prefs)

In [3]:
# 使用chromedriver才可以用开发者权限
chrome_driver_path = ".//chrome_driver//chromedriver.exe"
browser = webdriver.Chrome(chrome_driver_path, chrome_options=options)

In [4]:
browser.get('http://portal.yw56.com.cn/login')

In [5]:
yw_account = 401614
yw_psw = 401614

In [6]:
def find_elements_by_css_selector(class_name, order_num):
    class_name = class_name.replace(" ", ".")
    class_name = '.' + class_name
    return browser.find_elements_by_css_selector(class_name)[order_num]

In [7]:
yw_account_input = find_elements_by_css_selector('form-control form-control-solid placeholder-no-fix', 0)
yw_psw_input = find_elements_by_css_selector('form-control form-control-solid placeholder-no-fix', 1)

In [8]:
# 输入账户密码登录
yw_account_input.click()
yw_account_input.send_keys(yw_account)
yw_psw_input.click()
yw_psw_input.send_keys(yw_psw)
time.sleep(10)

In [9]:
# 登录
login_button = find_elements_by_css_selector("btn btn-block ladda-button", 0)
login_button.click()
time.sleep(10)

In [10]:
# 点击我的历史账单
browser.get('http://portal.yw56.com.cn/bill/index?type=1')

In [11]:
# 选择‘账单’
# https://stackoverflow.com/questions/7867537/selenium-python-drop-down-menu-option-value
browser.find_element_by_xpath("//select[@name='transType']/option[text()='账单']").click()

In [12]:
# 日期选择器
import datetime as dt
from datetime import timedelta
today_date = dt.datetime.today()
ninety_days_ago = today_date - timedelta(days=120)
ninety_days_ago_str_format = ninety_days_ago.strftime('%Y-%m-%d')
# ninety_days_ago_str_format

In [13]:
# 设置日期
start_date = find_elements_by_css_selector('form-control', 0)
start_date.clear()
start_date.send_keys(ninety_days_ago_str_format)

In [14]:
# 点击查询
search_bill = find_elements_by_css_selector("btn", 0)
search_bill.click()
time.sleep(10)

In [15]:
# 设置条目数
num_of_rows = find_elements_by_css_selector('btn-group dropup', 0)
num_of_rows.click()

# 点击最大条目数
# https://stackoverflow.com/questions/34583524/python-selenium-click-on-all-li-in-ul
# https://stackoverflow.com/questions/18026766/selenium-select-element-with-multiple-attributes
num_of_max_rows = browser.find_elements_by_xpath("//ul[@class = 'dropdown-menu'][@role = 'menu']//li[not(@class)]")
# num_of_max_rows[-1].text
num_of_max_rows[-1].click()
time.sleep(5)

In [16]:
# 查看有多少条记录
# https://stackoverflow.com/questions/14831041/how-to-count-no-of-rows-in-table-from-web-application-using-selenium-python-webd
row_count = len(browser.find_elements_by_xpath("//table[@id='table']/tbody/tr"))
print(row_count)
time.sleep(5)

50


In [17]:
# 点击每条账单
'''
bill_id = 0
bill_location = browser.find_elements_by_xpath("//table[@id='table']/tbody/tr[@data-index='" + str(bill_id) + "']/td")[5]
bill_html = "http://portal.yw56.com.cn" + bill_location.get_attribute('innerHTML').split('"')[1].replace('amp;','')
browser.get(bill_html)
'''

'\nbill_id = 0\nbill_location = browser.find_elements_by_xpath("//table[@id=\'table\']/tbody/tr[@data-index=\'" + str(bill_id) + "\']/td")[5]\nbill_html = "http://portal.yw56.com.cn" + bill_location.get_attribute(\'innerHTML\').split(\'"\')[1].replace(\'amp;\',\'\')\nbrowser.get(bill_html)\n'

In [18]:
# 点击下载
'''
bill_download = find_elements_by_css_selector("btn", 1)
bill_download.click()
'''

'\nbill_download = find_elements_by_css_selector("btn", 1)\nbill_download.click()\n'

In [19]:
for i in range(row_count):
    # 点击我的历史账单
    browser.get('http://portal.yw56.com.cn/bill/index?type=1')
    # 选择‘账单’
    # https://stackoverflow.com/questions/7867537/selenium-python-drop-down-menu-option-value
    browser.find_element_by_xpath("//select[@name='transType']/option[text()='账单']").click()
    # 日期选择器
    today_date = dt.datetime.today()
    ninety_days_ago = today_date - timedelta(days=90)
    ninety_days_ago_str_format = ninety_days_ago.strftime('%Y-%m-%d')
    # ninety_days_ago_str_format
    # 设置日期
    start_date = find_elements_by_css_selector('form-control', 0)
    start_date.clear()
    start_date.send_keys(ninety_days_ago_str_format)
    # 点击查询
    search_bill = find_elements_by_css_selector("btn", 0)
    search_bill.click()
    time.sleep(10)
    # 设置条目数
    num_of_rows = find_elements_by_css_selector('btn-group dropup', 0)
    num_of_rows.click()
    # 点击最大条目数
    # https://stackoverflow.com/questions/34583524/python-selenium-click-on-all-li-in-ul
    # https://stackoverflow.com/questions/18026766/selenium-select-element-with-multiple-attributes
    num_of_max_rows = browser.find_elements_by_xpath("//ul[@class = 'dropdown-menu'][@role = 'menu']//li[not(@class)]")
    # num_of_max_rows[-1].text
    num_of_max_rows[-1].click()
    time.sleep(5)
    # 点击每条账单
    bill_id = i
    try:
        bill_location = browser.find_elements_by_xpath("//table[@id='table']/tbody/tr[@data-index='" + str(bill_id) + "']/td")[5]
        print(bill_location)
        bill_html = "http://portal.yw56.com.cn" + bill_location.get_attribute('innerHTML').split('"')[1].replace('amp;','')
        print(bill_html)
        browser.get(bill_html)
        # 点击下载
        bill_download = find_elements_by_css_selector("btn", 1)
        bill_download.click()
        time.sleep(5)
    except Exception as err:
        print('Exception occurred: ' + str(err))

<selenium.webdriver.remote.webelement.WebElement (session="eba6cf31143ff9ad74baf0328a6116ed", element="696b9d5f-d112-42df-b080-6116f3a84685")>
Exception occurred: list index out of range
<selenium.webdriver.remote.webelement.WebElement (session="eba6cf31143ff9ad74baf0328a6116ed", element="c157a19f-c27a-43e6-a54b-745465d53642")>
http://portal.yw56.com.cn/bill/index?type=3&mainNo=TC01_2019-09-36_2019-09-36_04_401614_CNY&startDate=2019-09-02&endDate=2019-09-08&changeStatus=0&transType=TC01&billAmount=3008.41&rowNum=2
<selenium.webdriver.remote.webelement.WebElement (session="eba6cf31143ff9ad74baf0328a6116ed", element="c633025f-e15d-4726-bb8c-5dc5ecfe107b")>
http://portal.yw56.com.cn/bill/index?type=3&mainNo=TC02_2019-09-36_2019-08-32_04_401614_CNY_0&startDate=2019-09-02&endDate=2019-09-08&changeStatus=1&transType=TC02&billAmount=-35.14&rowNum=3
<selenium.webdriver.remote.webelement.WebElement (session="eba6cf31143ff9ad74baf0328a6116ed", element="7eade35a-7943-4f58-9021-22c3b53a2c56")>
Exc

<selenium.webdriver.remote.webelement.WebElement (session="eba6cf31143ff9ad74baf0328a6116ed", element="bd00f349-d888-4087-8e30-629674fd7529")>
http://portal.yw56.com.cn/bill/index?type=3&mainNo=TC02_2019-06-25_2019-05-21_04_401614_CNY_0&startDate=2019-06-17&endDate=2019-06-23&changeStatus=1&transType=TC02&billAmount=-49.26&rowNum=33
<selenium.webdriver.remote.webelement.WebElement (session="eba6cf31143ff9ad74baf0328a6116ed", element="a742370a-398a-4b6f-ae98-1b37f10fb2a8")>
http://portal.yw56.com.cn/bill/index?type=3&mainNo=TC01_2019-06-25_2019-06-25_04_401614_CNY&startDate=2019-06-17&endDate=2019-06-23&changeStatus=0&transType=TC01&billAmount=4380.81&rowNum=34
<selenium.webdriver.remote.webelement.WebElement (session="eba6cf31143ff9ad74baf0328a6116ed", element="7f3479ff-4c0f-4bb1-8b66-ed21a8747e3d")>
Exception occurred: list index out of range
<selenium.webdriver.remote.webelement.WebElement (session="eba6cf31143ff9ad74baf0328a6116ed", element="e435da20-5174-48b7-8d23-be958dfb15ef")>
E